https://www.youtube.com/watch?v=81S01c9zytE

Functions as object

In [1]:
fruit = ['banana','grapes','lime','pineapple']
sorted(fruit,key = len) #passing function as another function

['lime', 'banana', 'grapes', 'pineapple']

In [2]:
?sorted

In [3]:
def fibonacci(n):
    """create a list of n fibonacci numbers"""
    a = 0
    b = 1
    l = [0]
    for _ in range(n):
        a,b = b,a+b
        l.append(a)
    return l

In [4]:
fibonacci(10)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [5]:
fibonacci.__doc__ #being a function it has attributes

'create a list of n fibonacci numbers'

In [6]:
fibonacci.__annotations__.values

<function dict.values>

In [7]:
fibonacci.__code__.co_varnames

('n', 'a', 'b', 'l', '_')

In [8]:
from inspect import signature
signature(fibonacci).parameters

mappingproxy({'n': <Parameter "n">})

Decorators are syntactic sugar

function replacement

In [9]:
def deco(f):
    def inner():
        return "inner result"
    return inner

@deco
def target():
    return "original result"

In [10]:
target()

'inner result'

In [11]:
target #function as we can see is actually replaced

<function __main__.deco.<locals>.inner>

Decorators run at import time. Only importing will run all the function inside

In [12]:
import dis

In [13]:
dis.dis(square)

NameError: name 'square' is not defined

In [14]:
b = 6

def f1():
    a = 5
    print(a)
    print(b)

dis.dis(f1)

  4           0 LOAD_CONST               1 (5)
              3 STORE_FAST               0 (a)

  5           6 LOAD_GLOBAL              0 (print)
              9 LOAD_FAST                0 (a)
             12 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             15 POP_TOP

  6          16 LOAD_GLOBAL              0 (print)
             19 LOAD_GLOBAL              1 (b)
             22 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             25 POP_TOP
             26 LOAD_CONST               0 (None)
             29 RETURN_VALUE


In [15]:
#same but a little twist
b = 6

def f2():
    a = 5
    print(a)
    print(b)
    b = 4

dis.dis(f2)

  5           0 LOAD_CONST               1 (5)
              3 STORE_FAST               0 (a)

  6           6 LOAD_GLOBAL              0 (print)
              9 LOAD_FAST                0 (a)
             12 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             15 POP_TOP

  7          16 LOAD_GLOBAL              0 (print)
             19 LOAD_FAST                1 (b)
             22 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             25 POP_TOP

  8          26 LOAD_CONST               2 (4)
             29 STORE_FAST               1 (b)
             32 LOAD_CONST               0 (None)
             35 RETURN_VALUE


In [16]:
#same but a little twist
b = 6

def f3():
    a = 5
    global b
    print(a)
    print(b)
    b = 4

dis.dis(f3)

  5           0 LOAD_CONST               1 (5)
              3 STORE_FAST               0 (a)

  7           6 LOAD_GLOBAL              0 (print)
              9 LOAD_FAST                0 (a)
             12 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             15 POP_TOP

  8          16 LOAD_GLOBAL              0 (print)
             19 LOAD_GLOBAL              1 (b)
             22 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             25 POP_TOP

  9          26 LOAD_CONST               2 (4)
             29 STORE_GLOBAL             1 (b)
             32 LOAD_CONST               0 (None)
             35 RETURN_VALUE


### Class analogy first works similar to the closure

In [17]:
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self,new_value):
        self.series.append(new_value)
        return sum(self.series)/len(self.series)


In [18]:
avg = Averager()  #avg is an object of class Averager
avg(10)   #avg object is used as a function

10.0

In [19]:
avg(11)

10.5

In [20]:
avg(12)

11.0

### Now to real closure - this is in functions instead of class

In [21]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)  #here series is a free variable
        return sum(series)/len(series)
    return averager

In [22]:
avg = make_averager()

In [23]:
avg(10)

10.0

In [24]:
avg(11)

10.5

In [25]:
avg(12)

11.0

In [26]:
avg.__closure__

(<cell at 0x0000022F6DBADA08: list object at 0x0000022F6DC68A48>,)

In [27]:
avg.__closure__[0].cell_contents #only one closure element as only one free variable

[10, 11, 12]

In [28]:
avg.__code__.co_varnames #only one local variable

('new_value',)

In [29]:
avg.__code__.co_freevars

('series',)

In [30]:
def make_averager2():
    total = 0
    count = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [31]:
avg = make_averager2()
avg(10) #to resolve this problem

UnboundLocalError: local variable 'count' referenced before assignment

In [32]:
avg.__closure__ #there is no closure defined here

In [33]:
#way around for that is 

def make_averager3():
    total = 0
    count = 0
    def averager(new_value):
        nonlocal total,count 
        count += 1
        total += new_value
        return total/count
    return averager

In [34]:
avg = make_averager3()
avg(10)

10.0

In [35]:
avg(11)

10.5

In [36]:
avg.__closure__ #2 closure values

(<cell at 0x0000022F6DBAD348: int object at 0x000000005F910210>,
 <cell at 0x0000022F6DBAD768: int object at 0x000000005F910470>)

In [37]:
avg.__code__.co_freevars

('count', 'total')

In [38]:
#way around for that is 

def make_averager4():
    total = 0
    count = 0
    def averager(new_value):
        global total,count 
        count += 1
        total += new_value
        return total/count
    return averager

In [39]:
avg = make_averager4() # global is of no use

In [40]:
avg(10)

NameError: name 'count' is not defined

In [41]:
avg.__closure__

### Simplest decorator example

In [42]:
def floatify(f):
    
    def floated(n):
        result = f(n)
        return float(result)
    return floated # change the function 
        

In [43]:
@floatify
def square(n):
    return n * n

In [44]:
2 * 2

4

In [45]:
square(2) # 4.0 because of floatify

4.0

In [46]:
square

<function __main__.floatify.<locals>.floated>

### Clock Decorator

In [47]:
from functools import wraps
import time

def clock(f):
    
    @wraps(f) #this is wraps to preserve the metadata 
    def clocked(*args):
        start = time.time()
        time.sleep(0.1) #added this to see the time diff
        result = f(*args) 
        print("time taken is {0:.16f}".format(time.time() - start))
        return result
    return clocked

In [48]:
#this is copied from above
@clock
def fibonacci(n):
    """create a list of n fibonacci numbers"""
    a = 0
    b = 1
    l = [0]
    for _ in range(n):
        a,b = b,a+b
        l.append(a)
    return l

In [49]:
#now first thing to notice is that the fibonacci will have its signature
fibonacci  #this is different from above in case of square

<function __main__.fibonacci>

In [50]:
fibonacci.__doc__

'create a list of n fibonacci numbers'

In [51]:
fibonacci(10)

time taken is 0.1028015613555908


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [52]:
@clock
def factorial(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n - 1)

In [53]:
factorial(10)

time taken is 0.1027216911315918
time taken is 0.2183299064636230
time taken is 0.3289558887481689
time taken is 0.4384553432464600
time taken is 0.5414764881134033
time taken is 0.6488864421844482
time taken is 0.7546453475952148
time taken is 0.8572158813476562
time taken is 0.9614126682281494
time taken is 1.0629723072052002


3628800

### Parametrized Decorators -- Also Decorator Factories

In [54]:
def clock(fmt = 'the time taken is {0:.16f}'):
    def decorator(f):
        
        @wraps(f) #this is wraps to preserve the metadata 
        def clocked(*args):
            start = time.time()
            time.sleep(0.1) #added this to see the time diff
            result = f(*args) 
            print(fmt.format(time.time() - start))
            return result
        return clocked
    return decorator

In [55]:
@clock()
def factorial(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n - 1)

In [56]:
factorial(10)

the time taken is 0.1022274494171143
the time taken is 0.2049131393432617
the time taken is 0.3074047565460205
the time taken is 0.4075324535369873
the time taken is 0.5076577663421631
the time taken is 0.6231558322906494
the time taken is 0.7289111614227295
the time taken is 0.8385748863220215
the time taken is 0.9480381011962891
the time taken is 1.0583782196044922


3628800

In [57]:
@clock('the time taken for factorial function is {0:.2f}')
def factorial1(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n - 1)

In [58]:
factorial1(20)

the time taken is 0.1001558303833008
the time taken is 0.2157502174377441
the time taken is 0.3313729763031006
the time taken is 0.4469432830810547
the time taken is 0.5627486705780029
the time taken is 0.6783220767974854
the time taken is 0.7786762714385986
the time taken is 0.8987438678741455
the time taken is 0.9988899230957031
the time taken is 1.1124677658081055
the time taken is 1.2161564826965332
the time taken is 1.3185739517211914
the time taken is 1.4186859130859375
the time taken is 1.5287172794342041
the time taken is 1.6346576213836670
the time taken is 1.7346692085266113
the time taken is 1.8442931175231934
the time taken is 1.9579501152038574
the time taken is 2.0591943264007568
the time taken for factorial function is 2.16


2432902008176640000

### LRU cache

In [59]:
print ('hello','{0:.6f}'.format(636736.73783))

hello 636736.737830


In [60]:
def clock(fmt = 'the time taken is {0:.16f}'):
    def decorator(f):
        
        @wraps(f) #this is wraps to preserve the metadata 
        def clocked(*args):
            start = time.time()
            time.sleep(0.1) #added this to see the time diff
            result = f(*args) 
            #my assumption for now is only one argument we can change it though
            print('func',f.__name__,'(',args[0],') ->',result,fmt.format(time.time() - start))
            return result
        return clocked
    return decorator

In [61]:
@clock('time taken is : {0:.10f}')
#returning nth fibonacci number
def fibonacci_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

In [62]:
fibonacci_recursive(8)

func fibonacci_recursive ( 2 ) -> 1 time taken is : 0.1155126095
func fibonacci_recursive ( 1 ) -> 0 time taken is : 0.1003522873
func fibonacci_recursive ( 3 ) -> 1 time taken is : 0.3200340271
func fibonacci_recursive ( 2 ) -> 1 time taken is : 0.1214566231
func fibonacci_recursive ( 4 ) -> 2 time taken is : 0.5416402817
func fibonacci_recursive ( 2 ) -> 1 time taken is : 0.1001241207
func fibonacci_recursive ( 1 ) -> 0 time taken is : 0.1115577221
func fibonacci_recursive ( 3 ) -> 1 time taken is : 0.3164005280
func fibonacci_recursive ( 5 ) -> 3 time taken is : 0.9669535160
func fibonacci_recursive ( 2 ) -> 1 time taken is : 0.1041531563
func fibonacci_recursive ( 1 ) -> 0 time taken is : 0.1154899597
func fibonacci_recursive ( 3 ) -> 1 time taken is : 0.3197317123
func fibonacci_recursive ( 2 ) -> 1 time taken is : 0.1001336575
func fibonacci_recursive ( 4 ) -> 2 time taken is : 0.5199916363
func fibonacci_recursive ( 6 ) -> 5 time taken is : 1.5899155140
func fibonacci_recursive 

13

In [63]:
fibonacci(8) #correct we are evaluating it as 7

time taken is 0.1038780212402344


[0, 1, 1, 2, 3, 5, 8, 13, 21]

In [64]:
#magic of lru_cache
from functools import lru_cache

@lru_cache()
@clock('time taken is : {0:.10f}')
#returning nth fibonacci number
def fibonacci_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

In [65]:
#no need of memoization
fibonacci_recursive(8) # cool 1 only called once

func fibonacci_recursive ( 2 ) -> 1 time taken is : 0.1002490520
func fibonacci_recursive ( 1 ) -> 0 time taken is : 0.1040802002
func fibonacci_recursive ( 3 ) -> 1 time taken is : 0.3057212830
func fibonacci_recursive ( 4 ) -> 2 time taken is : 0.4275910854
func fibonacci_recursive ( 5 ) -> 3 time taken is : 0.5402431488
func fibonacci_recursive ( 6 ) -> 5 time taken is : 0.6437354088
func fibonacci_recursive ( 7 ) -> 8 time taken is : 0.7558131218
func fibonacci_recursive ( 8 ) -> 13 time taken is : 0.8712155819


13

In [66]:
repr(fibonacci_recursive)

'<functools._lru_cache_wrapper object at 0x0000022F6DC3C978>'

In [67]:
who

Averager	 avg	 b	 clock	 deco	 dis	 f1	 f2	 f3	 
factorial	 factorial1	 fibonacci	 fibonacci_recursive	 floatify	 fruit	 lru_cache	 make_averager	 make_averager2	 
make_averager3	 make_averager4	 signature	 square	 target	 time	 wraps	 


### Decorators in Class

In [68]:
a = 15
print (repr(a))

15


In [69]:
class Clocker:
    
    def __init__(self,fmt = '[{end:0.8f}s] {name} ({args}) -> {result}'):
        self.fmt = fmt
    
    def __call__(self,func):
        @wraps(func)
        def clocked(*_args):
            start = time.time()
            _result = func(*_args)
            end = time.time() - start
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

In [70]:
@Clocker()
def fibonacci_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

In [71]:
fibonacci_recursive(10)

[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (1) -> 0
[0.00000000s] fibonacci_recursive (3) -> 1
[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (4) -> 2
[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (1) -> 0
[0.00000000s] fibonacci_recursive (3) -> 1
[0.00000000s] fibonacci_recursive (5) -> 3
[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (1) -> 0
[0.00000000s] fibonacci_recursive (3) -> 1
[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (4) -> 2
[0.00000000s] fibonacci_recursive (6) -> 5
[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (1) -> 0
[0.00000000s] fibonacci_recursive (3) -> 1
[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (4) -> 2
[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (1) -> 0
[0.00000000s] fibonacci_recursive (3) -> 1
[0.00000000

34

In [72]:
fibonacci_recursive

<function __main__.fibonacci_recursive>

In [73]:
fibonacci_recursive.__doc__

In [74]:
@lru_cache()
@Clocker()
def fibonacci_recursive(n):
    '''this function is finding the nth fibonacci number using recursion'''
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

fibonacci_recursive(10)

[0.00000000s] fibonacci_recursive (2) -> 1
[0.00000000s] fibonacci_recursive (1) -> 0
[0.00000000s] fibonacci_recursive (3) -> 1
[0.00000000s] fibonacci_recursive (4) -> 2
[0.00000000s] fibonacci_recursive (5) -> 3
[0.00000000s] fibonacci_recursive (6) -> 5
[0.00000000s] fibonacci_recursive (7) -> 8
[0.00000000s] fibonacci_recursive (8) -> 13
[0.00000000s] fibonacci_recursive (9) -> 21
[0.00000000s] fibonacci_recursive (10) -> 34


34

In [75]:
fibonacci_recursive

In [76]:
fibonacci_recursive.__doc__

'this function is finding the nth fibonacci number using recursion'

### @property 

In [77]:
class Employee:
    
    def __init__(self,first,last):
        self.first = first
        self.last = last
        
    @property
    def email(self):
        return '{}.{}@email.sc.edu'.format(self.first,self.last)
    
    @property
    def full_name(self):
        return '{} {}'.format(self.first,self.last)
    
emp1 = Employee('vijendra','rana')

In [78]:
emp1.email

'vijendra.rana@email.sc.edu'

In [79]:
emp1.full_name

'vijendra rana'

In [80]:
emp1.full_name = 'ajay rana'

AttributeError: can't set attribute

https://www.youtube.com/watch?v=jCzT9XFZ5bw&list=PL-osiE80TeTsqhIuOqKhwlXsIBIdSeYtc&index=6&t=312s

In [81]:

class Employee:
    
    def __init__(self,first,last):
        self.first = first
        self.last = last
        
    @property
    def email(self):
        return '{}.{}@email.sc.edu'.format(self.first,self.last)
    
    @property
    def full_name(self):
        return '{} {}'.format(self.first,self.last)
    
    @full_name.setter
    def full_name(self,name):
        first,last = name.split(' ')
        self.first = first
        self.last = last
    
    @email.setter
    def email(self,new_email):
        first,last = new_email.split('@')[0].split('.')
        self.first = first
        self.last = last
    
    @email.deleter
    def email(self):
        print ("Deleting the email hence the first name and last name as well")
        self.first = None
        self.last = None
        
emp1 = Employee('vijendra','rana')

In [82]:
emp1.email = 'vijen.1991@gmail.com'

In [83]:
emp1.email

'vijen.1991@email.sc.edu'

In [84]:
emp1.full_name = 'Vijendra Rana'

In [85]:
emp1.email

'Vijendra.Rana@email.sc.edu'

In [86]:
emp1.full_name

'Vijendra Rana'

In [87]:
emp1.first

'Vijendra'

In [88]:
emp1.last

'Rana'

In [89]:
del emp1.full_name # can't delete this

AttributeError: can't delete attribute

In [90]:
del emp1.email

Deleting the email hence the first name and last name as well


In [91]:
print(emp1.first)

None


In [92]:
print(emp1.email)

None.None@email.sc.edu


In [93]:
print(emp1.last)

None


In [94]:
print(emp1.full_name)

None None


In [95]:
import memory_profiler

In [104]:
print (memory_profiler.memory_usage()[0],'MB')

50.75 MB
